## Bounded arrays in C

As we have seen, arrays in C are implemented in the most simple way possible.  They are simply represended as a memory address with no start of end indices, no bounds checking.  Only very simple math is used to calculate the positions of elements within the array (since the compiler knows the size (width) of each element in the array).

In this example we will go over a better DIY array representation.  These arrays will have arbitrary element indices and include bounds checking.

The purpose of this exercise is two fold:
1) to show how you might go about implementing your own better arrays, and
2) to exercise our use of pointers.

## Specifics
Our arrays will have a user-specified start index (it doesn't have to be zero).  We only require that it is an integer.

Our arrays will have a user-specified number of elements.  Any attempt to read or write outside of the valid bounds of the array will be prevented and instead an error code and error message will be generated.

## Storage
We will store our arrays as a structure as follows.

In [1]:
%%file barray.h
#ifndef BARRAY_H
#define BARRAY_H

struct barray_struct
{
  void *memptr;    /* pointer to the location in memory where the data of the 
                      array is stored */    
  int start_index; /* lowest index permitted */
  int element_size;/* size of each element */
  int n_elements;  /* number of elements in the array */
};

struct barray_struct *new_barray( int start_index, int n_elements, int element_size );
/* This function will return a pointer to a newly allocated barray_struct structure,
   or NULL if the call to malloc fails.  The start_index is the lowest index permitted.
   n_elements is the number of elements in the array.  element_size is the size of one
   element.
    
   E.g.
   struct barray_struct *int_array;
   int_array = new_barray( 1, 10, sizeof(int) );

   Would make int_array point to a new barray_struct structure that can store
   10 elements of type integer, with indices 1 to 10.
*/

int set_item( struct barray_struct *barray, int index, void *value_ptr );
/* This function will return the value 0 if the index is valid, and the value -1 if 
   the index is invalid.  Additionally, it will print an error message to stderr if 
   the index is not valid.

   If the index is valid, this function will set the value of the element given by 
   index in the array pointed to by barray to to the value pointed to by value_ptr.
*/

int get_item( struct barray_struct *barray, int index, void *value_ptr );
/* This function will return the value 0 if the index is valid, and the value -1 if 
   the index is invalid.  Additionally, it will print an error message to stderr if 
   the index is not valid.

   If the index is valid, this function will retreive the value of the element given by 
   index in the array pointed to by barray, and store it at the memory location pointed 
   to by value_ptr.
*/

#endif

Overwriting barray.h


In [2]:
%%file ex05main.c
#include <stdio.h>
#include "barray.h"

int main()
{
    int i;
    int value;
    
    struct barray_struct *barray;
    
    barray = new_barray( -3, 14, sizeof(int) ); /* create array of integers with indices
                                                   from -3 to 10 */
    
    for (i=-3;i<=10;i++)
    {
        value = i*7;  /* store the value of the index multiplied by 7 at each location */
        set_item( barray, i, &value );
    }
    
    /* try to set an invalid index */
    value = 99;
    set_item( barray, 11, &value );
    
    /* try to get an invalid index */
    get_item( barray, -4, &value );
    
    for (i=-3;i<=10;i++)
    {
        get_item( barray, i, &value );
        printf( "%+02d: %+02d\n", i, value );
    }
    
    return 0;
}

Overwriting ex05main.c


In [3]:
print( "\t" );


In [4]:
%%file makefile
ex05: ex05main.o ex05new_barray.o ex05set_item.o ex05get_item.o
	gcc -ansi -Wall -pedantic ex05main.o ex05new_barray.o ex05set_item.o \
    ex05get_item.o -o ex05
    
ex05main.o: ex05main.c barray.h
	gcc -ansi -Wall -pedantic -c ex05main.c -o ex05main.o
    
ex05new_barray.o: ex05new_barray.c barray.h
	gcc -ansi -Wall -pedantic -c ex05new_barray.c -o ex05new_barray.o

ex05set_item.o: ex05set_item.c barray.h
	gcc -ansi -Wall -pedantic -c ex05set_item.c -o ex05set_item.o

ex05get_item.o: ex05get_item.c barray.h
	gcc -ansi -Wall -pedantic -c ex05get_item.c -o ex05get_item.o
    
clean:
	rm *.o ex05


Overwriting makefile


In [5]:
%%file ex05new_barray.c
#include "barray.h"
#include <stdlib.h>


struct barray_struct *new_barray( int start_index, int n_elements, int element_size )
/* This function will return a pointer to a newly allocated barray_struct structure,
   or NULL if the call to malloc fails.  The start_index is the lowest index permitted.
   n_elements is the number of elements in the array.  element_size is the size of one
   element.
    
   E.g.
   struct barray_struct *int_array;
   int_array = new_barray( 1, 10, sizeof(int) );

   Would assign make int_array point to a new barray_struct structure that can store
   10 elements of type integer, with indices 1 to 10.
*/
{
    /* add code here */
    struct barray_struct *barray;

    barray = malloc( sizeof(struct barray_struct) );
    (*barray).start_index = start_index;
    (*barray).n_elements = n_elements;
    (*barray).element_size = element_size;
    (*barray).memptr = malloc( element_size * n_elements );

    
    return barray;
}


Overwriting ex05new_barray.c


In [6]:
%%file ex05set_item.c
#include "barray.h"
#include <stdio.h>
#include <stdlib.h>
#include <string.h>


int set_item( struct barray_struct *barray, int index, void *value_ptr )
/* This function will return the value 0 if the index is valid, and the value -1 if 
   the index is invalid.  Additionally, it will print an error message to stderr if 
   the index is not valid.

   If the index is valid, this function will set the value of the element given by 
   index in the array pointed to by barray to to the value pointed to by value_ptr.
*/
{
    char *memptr;
    int array_index;
    
    /* add code here */
    if ( (index >= barray->start_index) && (index < barray->n_elements + barray->start_index) )
    {
        /* valid index */
        memptr = barray->memptr;
        array_index = (index - barray->start_index)*barray->element_size;
        memcpy( memptr+array_index, value_ptr, barray->element_size );
        return 0;
    }
    else
    {
        /* invalid index */
        fprintf( stderr, "Erorr:  bad index %d (start_index=%d, n_elements=%d)\n",
                                      index, barray->start_index, barray->n_elements );
        return -1;
    }
    

}

Overwriting ex05set_item.c


In [7]:
%%file ex05get_item.c
#include "barray.h"
#include <string.h>
#include <stdlib.h>
#include <stdio.h>


int get_item( struct barray_struct *barray, int index, void *value_ptr )
/* This function will return the value 0 if the index is valid, and the value -1 if 
   the index is invalid.  Additionally, it will print an error message to stderr if 
   the index is not valid.

   If the index is valid, this function will retreive the value of the element given by 
   index in the array pointed to by barray, and store it at the memory location pointed 
   to by value_ptr.
*/
{
    /* add code here */
    char *memptr;
    int array_index;
    
    /* add code here */
    if ( (index >= barray->start_index) && (index < barray->n_elements + barray->start_index) )
    {
        /* valid index */
        memptr = barray->memptr;
        array_index = (index - barray->start_index)*barray->element_size;
        memcpy( value_ptr, memptr+array_index, barray->element_size );
        return 0;
    }
    else
    {
        /* invalid index */
        fprintf( stderr, "Erorr:  bad index %d (start_index=%d, n_elements=%d)\n",
                                      index, barray->start_index, barray->n_elements );
        return -1;
    }

}

Overwriting ex05get_item.c


In [8]:
%%bash
make clean

rm *.o ex05


In [1]:
%%bash
make

gcc -Wall -pedantic -Ilab04 -ansi -c lab04/main.c -o lab04/objFiles/main.o
makefile:8: recipe for target 'main' failed


gcc: error: lab04/main.c: No such file or directory
gcc: fatal error: no input files
compilation terminated.
make: *** [main] Error 1


In [10]:
%%bash
./ex05

-3: -21
-2: -14
-1: -7
+0: +0
+1: +7
+2: +14
+3: +21
+4: +28
+5: +35
+6: +42
+7: +49
+8: +56
+9: +63
+10: +70


Erorr:  bad index 11 (start_index=-3, n_elements=14)
Erorr:  bad index -4 (start_index=-3, n_elements=14)
